Ensuring Security of Cloud Server Instances
========================

@DanDye; MIL-OSS Hackathon, Tampa, FL October 23rd - 25th 2015



First thought, get content of config files...

In [178]:
"""dump config files
============
package-name
============
+++++++++++++++++
config file's SHA
+++++++++++++++++
-----------------
config file's loc
-----------------
contents
++++++++++++++++++++++
next config file's SHA
++++++++++++++++++++++
----------------------
next config file's loc
----------------------
next file's contents
...
"""
decorate_string = lambda stringin, decorator: ((decorator * len(stringin)) + "\n"
                                               + stringin + "\n"
                                               + (decorator * len(stringin)))    

concerns = ["openssh-server", "some-other-package", ] # List of packages to dump
                     
for concern in concerns:
    print decorate_string(concern, "=")
    config_files = !dpkg-query --showformat='${Conffiles}\n' --show openssh-server # FixMe: hard-coded
    for aline in config_files:
        _,apath,sha = aline.split(" ")
        print decorate_string(sha,"+")
        print decorate_string(apath,"-")
        !cat "$apath"       

openssh-server
++++++++++++++++++++++++++++++++
91e764631983aab83a9e08bd9e00a9f9
++++++++++++++++++++++++++++++++
-----------------------------------
/etc/network/if-up.d/openssh-server
-----------------------------------
#! /bin/sh
# Reload the OpenSSH server when an interface comes up, to allow it to start
# listening on new addresses.

set -e

# Don't bother to restart sshd when lo is configured.
if [ "$IFACE" = lo ]; then
	exit 0
fi

# Only run from ifup.
if [ "$MODE" != start ]; then
	exit 0
fi

# OpenSSH only cares about inet and inet6. Get ye gone, strange people
# still using ipx.
if [ "$ADDRFAM" != inet ] && [ "$ADDRFAM" != inet6 ]; then
	exit 0
fi

# Is /usr mounted?
if [ ! -e /usr/sbin/sshd ]; then
	exit 0
fi

if [ ! -f /var/run/sshd.pid ] || \
   [ "$(ps -p "$(cat /var/run/sshd.pid)" -o comm=)" != sshd ]; then
	exit 0
fi

# We'd like to use 'reload' here, but it has some problems; see
# #502444.
invoke-rc.d ssh restart >/dev/null 2>&1 || true

exit 0
+++++++++++++++++++++++

Now I need a checklist for these configuration files...

Extensible Configuration Checklist Description Format (aka XCCDF)
---------------------------------------------------------------------------
"XCCDF is a specification language for writing security checklists, benchmarks, and related kinds of documents."

http://scap.nist.gov/specifications/xccdf/


In [185]:
from IPython.display import IFrame
render = lambda url: IFrame(url, width=900, height=500)

#render('http://cve.mitre.org/index.html') # CVE docs
render("http://scap.nist.gov/specifications/xccdf/")

In [186]:
# Security Configuration Automation Protocol (SCAP)
#render("http://www.admin-magazine.com/Articles/Checking-Compliance-with-OpenSCAP")
#render("https://nvd.nist.gov/scapproducts.cfm")
#render("http://www.open-scap.org/page/Download")
#render("http://www.redhat.com/security/data/oval/")
render("http://scap.nist.gov/")

In [139]:
# Example usage of oscap utility
rhel6_scap_cmd = """oscap xccdf eval --profile stig-rhel6-server-upstream \
    --results /tmp/`hostname`-ssg-results.xml \
    --report /tmp/`hostname`-ssg-results.html \
    --cpe /usr/share/xml/scap/ssg/content/ssg-rhel6-cpe-dictionary.xml \
          /usr/share/xml/scap/ssg/content/ssg-rhel6-xccdf.xml"""


In [200]:
"""Instantiate a RHEL6 Public AMI for testing"""
from boto import ec2

AWS_KEYS_MILOSS="AWS_KEYS_MILOSS"
mykey = paramiko.RSAKey.from_private_key_file("/home/dandye/.ssh/AWS_KEYS_MILOSS.pem")
AWS_ID = open("/home/dandye/AWS_ID.txt").read().strip()
AWS_KEY = open("/home/dandye/AWS_KEY.txt").read().strip() # keeping secrets isn't easy

conn = ec2.connect_to_region("us-east-1",
                                  aws_access_key_id=AWS_ID,
                                  aws_secret_access_key=AWS_KEY)

['kernel', 'instance_profile', 'root_device_type', 'private_dns_name', '_state', 'group_name', 'public_dns_name', 'id', 'state_reason', 'monitored', 'item', 'subnet_id', 'block_device_mapping', 'platform', 'eventsSet', 'ebs_optimized', 'client_token', '_in_monitoring_element', 'virtualization_type', 'architecture', 'ramdisk', '_previous_state', 'tags', 'key_name', 'interfaces', 'image_id', 'reason', 'groups', 'spot_instance_request_id', 'monitoring', 'requester_id', 'ip_address', 'monitoring_state', '_placement', 'ami_launch_index', 'dns_name', 'region', 'launch_time', 'persistent', 'instance_type', 'connection', 'root_device_name', 'hypervisor', 'private_ip_address', 'vpc_id', 'product_codes']
ami-1643ff7e
54.211.212.156
m1.small
us-east-1d


In [ ]:
# uses conn from cell above
filters = {'instance-state-name' : 'running'}
reservation = conn.get_all_instances(filters=filters)[0]
instance = reservation.instances[0]  #.__dict__.keys()

print instance.__dict__.keys()
print instance.image_id
print instance.ip_address
print instance.instance_type
print instance.placement

# (in terminal)
>>> conn = boto.ec2.connect_to_region("us-east-2",
...                                   aws_access_key_id=AWS_ID,
...                                   aws_secret_access_key=AWS_KEY)
>>> 
>>> rhel6_amis = ["ami-94a3eefc",]
>>> ami = rhel6_amis[0]
>>> conn = boto.ec2.connect_to_region("us-east-1",
...                                   aws_access_key_id=AWS_ID,
...                                   aws_secret_access_key=AWS_KEY)
>>> conn
EC2Connection:ec2.us-east-1.amazonaws.com
>>> conn.run_instances(ami, key_name="AWS_KEYS_MILOSS")
Reservation:r-5912128f
>>> reservations = conn.get_all_reservations()
>>> reservations
[Reservation:r-5912128f]
>>> instances = reservations[0].instances
>>> instances
[Instance:i-05396cba]
>>> inst = instances[0]
>>> inst.instance_type
u'm1.small'
>>> inst.placement
u'us-east-1d'
>>> from boto.manage.cmdshell import sshclient_from_instance


In [204]:
"""Setup some SSH/SFTP connex"""
import paramiko

server = instance.ip_address 
sshuser = 'ec2-user'

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # Fixes Server u'w.x.y.z' not found in known_hosts
client.load_system_host_keys()
client.connect(server, username=sshuser, pkey=mykey)

client

In [ ]:
"""Use Paramiko to run remote commands on Rackspace/EC2 Instance"""
def run(cmd):
    transport = client.get_transport()
    session = transport.open_session()
    session.set_combine_stderr(True)
    session.get_pty()
    session.exec_command("sudo {}".format(cmd))
    stdin = session.makefile('wb', -1)
    stdout = session.makefile('rb', -1)
    for line in stdout.read().splitlines():        
        print line

In [176]:
 run("sudo yum install -y openscap-utils scap-security-guide")
 run("rpm -ql scap-security-guide")

Loaded plugins: amazon-id, rhui-lb, security
Setting up Install Process
Package openscap-utils-1.0.10-3.el6.x86_64 already installed and latest version
Package scap-security-guide-0.1.21-3.el6.noarch already installed and latest version
Nothing to do
/usr/share/doc/scap-security-guide-0.1.21
/usr/share/doc/scap-security-guide-0.1.21/DISCLAIMER
/usr/share/doc/scap-security-guide-0.1.21/LICENSE
/usr/share/doc/scap-security-guide-0.1.21/rhel6-guide.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-cces.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-nistrefs-common.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-nistrefs.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-srgmap-flat.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-srgmap-flat.xhtml
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-srgmap.html
/usr/share/doc/scap-security-guide-0.1.21/table-rhel6-stig.html
/usr/share/man/en/man8/scap-security-guide.8.gz
/usr/share/scap-s

In [160]:
"""Use the XCCDF """
run("""oscap xccdf eval \
    --profile stig-rhel6-server-upstream \
    --results /tmp/ssg-results.xml \
    --report /tmp/ssg-results.html \
    --cpe /usr/share/xml/scap/ssg/content/ssg-rhel6-cpe-dictionary.xml \
          /usr/share/xml/scap/ssg/content/ssg-rhel6-xccdf.xml""")


#run("wget http://www.redhat.com/security/data/oval/Red_Hat_Enterprise_Linux_6.xml")

# DISA STIG
# http://iase.disa.mil/stigs/Documents/U_RedHat_6_V1R6_STIG.zip
#run("wget http://iasecontent.disa.mil/stigs/zip/Oct2015/U_RedHat_6_V1R9_STIG.zip")
#run("unzip U_RedHat_6_V1R9_STIG.zip")
#run("""sed -i 's/<Group\ \(.*\)/<Group\ selected="false"\ \1/g' U_RedHat_6_V1R9_STIG_Manual/U_RedHat_6_V1R9_Manual-xccdf.xml""")
#run("""oscap xccdf eval \
#      --profile MAC-1_Public \
#       --results MAC-1_Public_results.xml \
#       --cpe ./U_RedHat_6_V1R9_STIG_Manual/U_RedHat_6_V1R9_Manual-xccdf.xml""")

# United States Government Configuration Baseline (USGCB) on Red Hat Enterprise Linux 6
#run("""oscap xccdf eval \
#    --profile united_states_government_configuration_baseline \
#    --cpe usgcb-rhel5desktop-cpe-dictionary.xml \
#    --oval-results \
#    --fetch-remote-resources \
#    --results results.xml \
#    usgcb-rhel5desktop-xccdf.xml""")


Title
	Ensure /tmp Located On Separate Partition
Rule
	partition_for_tmp
Ident
	CCE-26435-8
Result
	fail

Title
	Ensure /var Located On Separate Partition
Rule
	partition_for_var
Ident
	CCE-26639-5
Result
	fail

Title
	Ensure /var/log Located On Separate Partition
Rule
	partition_for_var_log
Ident
	CCE-26215-4
Result
	fail

Title
	Ensure /var/log/audit Located On Separate Partition
Rule
	partition_for_var_log_audit
Ident
	CCE-26436-6
Result
	fail

Title
	Ensure /home Located On Separate Partition
Rule
	partition_for_home
Ident
	CCE-26557-9
Result
	fail

Title
	Encrypt Partitions
Rule
	encrypt_partitions
Ident
	CCE-27596-6
Result
	notchecked

Title
	Ensure Red Hat GPG Key Installed
Rule
	ensure_redhat_gpgkey_installed
Ident
	CCE-26506-6
Result
	pass

Title
	Ensure gpgcheck Enabled In Main Yum Configuration
Rule
	ensure_gpgcheck_globally_activated
Ident
	CCE-26709-6
Result
	pass

Title
	Ensure gpgcheck Enabled For All Yum Package Repositories
Rule
	ensure_gpgcheck_never_disabled
Ident
	C

In [165]:
# SCP the report
ami = "ami-94a3eefc" # FixMe: hard-coded
transport = paramiko.Transport((server, 22))  # server is IP of box
transport.connect(username="ec2-user", pkey=mykey)
sftp = paramiko.SFTPClient.from_transport(transport)
sftp.get("/tmp/ssg-results.html", "ami_{}_ssg_results.html".format(ami))
sftp.get("/tmp/ssg-results.xml", "ami_{}_ssg_results.xml".format(ami))

In [182]:
print ami
render("ami_{}_ssg_results.html".format(ami))

ami-94a3eefc


In [ ]:
amis = [ {"AMI_ID": "ami-2df5b444", "AMI_Name": "rhel6.4-oracle-ami"},
         {"AMI_ID": "ami-85e819ee", "AMI_Name": "redislabs-cluster-4.0.0-49-rhel6"},
         {"AMI_ID": "ami-dbb5c6b2", "AMI_Name": "NOW Developers RHEL6"},
       ]
        
        